In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
import importlib

In [6]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(train)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:08<01:16,  2.37it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [00:39<00:32,  3.00it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [00:52<00:43,  1.58it/s]

...

 79%|███████▉  | 158/200 [01:02<00:13,  3.10it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [01:20<00:00,  2.50it/s]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 11863.49it/s]


In [65]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [66]:
model = torch_classes.GRUNetV2(12,64).to(device='cuda:0')

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [40]:
def model_pipeline(trading_df:torch_classes.TradingData, config:dict):
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        model = torch_classes.GRUNetV2(12,hidden_size).to('cuda:0')
        config = wandb.config
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        trading_df.reset_hidden(config['hidden_size'])
        criterion = nn.SmoothL1Loss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [62]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [59]:
config_static = {'learning_rate':0.0001, 'hidden_size':64}

In [68]:
trading_data.reset_hidden(64)
trading_data.detach_hidden()

In [35]:
model

GRUNetV2(
  (gru): GRU(12, 64, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc0): Linear(in_features=12800, out_features=1024, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)

In [ ]:
stocks = [trading_data.stocksDict[x] for x in trading_data.stock_batches[0]]
X = trading_data.packed_val_x[0] 
Y = trading_data.packed_val_y[0].data

In [ ]:
# hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

In [ ]:
# hidden_in 

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')

In [38]:
# output,hidden = model(X,hidden_in,p1=True)

In [69]:
model = model_pipeline(trading_data, config_static)


  0%|          | 0/10000 [00:00<?, ?it/s]

{'epoch_loss': tensor(1.2450, device='cuda:0'), 'epoch_l1_loss': tensor(1.3581, device='cuda:0'), 'epoch': 0}
   stock  day  time    target      pred
0      0    0     0 -3.029704 -0.004441
1      1    0     0 -5.519986 -0.057125
2      2    0     0 -8.389950  0.010134
3      3    0     0 -4.010201  0.015898
4      4    0     0 -7.349849 -0.048142
5      5    0     0  6.779432 -0.031476
6      6    0     0 -2.499819 -0.017918
7      7    0     0 -1.959801 -0.036692
8      8    0     0 -5.970001 -0.000415
9      9    0     0  7.970333 -0.026925
created path
